In [1]:
#Import .ipynb as module: %run name.ipynb
import math
import random
import copy
import itertools as it

In [2]:
class Course:
    _staticId = 0
    def __init__(self, name, classes, semester):
        Course._staticId = Course._staticId + 1
        self.id       = Course._staticId
        self.name     = name
        self.classes  = classes
        self.semester = semester
        
    def __repr__(self):
        return 'Course {}: {} ({} classes, {}o semester)'.format(self.id, self.name, self.classes, self.semester)
    
class Program:
    def __init__(self, name, numSemesters, courses):
        self.name         = name
        self.numSemesters = numSemesters
        self.numCourses   = len(courses)
        self.courses      = courses
    
    def __repr__(self):
        return 'Program: {} ({} semesters, {} courses)'.format(self.name, self.numSemesters, self.numCourses)
    
class Schedule:
    def __init__(self):
        self.weekName = ['Seg', 'Ter', 'Qua', 'Qui', 'Sex']
        self.week = [[0]*2]*5
        
    def __repr__(self):
        strOut = '| '
        for i in self.weekName:
            strOut += i + '  '
        strOut += ' |\n| '
        for i in self.week:
            strOut += ' ' + str(i[0]) + '   '
        strOut += ' |\n| '
        for i in self.week:
            strOut += ' ' + str(i[1]) + '   '
        strOut += ' |\n'
        return strOut

In [3]:
courses1 = []

courses1.append(Course('Analise de Algoritmos', 2, 1))
courses1.append(Course('Sistemas Embarcados', 3, 1))
courses1.append(Course('Sistemas Operacionais', 2, 1))

print(courses1)

programs = []

programs.append(Program('EngComp', 10, courses1))

print(programs)

[Course 1: Analise de Algoritmos (2 classes, 1o semester), Course 2: Sistemas Embarcados (3 classes, 1o semester), Course 3: Sistemas Operacionais (2 classes, 1o semester)]
[Program: EngComp (10 semesters, 3 courses)]
